<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/17_Databricks_%26_Pyspark__Azure_Data_Lake_Storage_Integration_with_Databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Methods of ADLS Integration Databricks**

In [0]:
import findspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
storage_account = '*****'
access_key = '******************'
container_name = '*****'

#### Method 1: Using DLS access key directly

In [0]:
spark.conf.set(
    f'fs.azure.account.key.{storage_account}.dfs.core.windows.net',
    access_key
)

##### ls command (dbutils.fs.ls) from data lake azure
https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-utils#--file-system-utility-dbutilsfs

Lists the contents of a directory.

To display help for this command, run dbutils.fs.help("ls").

This example displays information about the contents of /tmp. The modificationTime field is available in Databricks Runtime 10.2 and above. In R, modificationTime is returned as a string.

In [0]:
dbutils.fs.ls(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/api=state/date=2024-04-11")


Show data json state and explode from ADLS

In [0]:
from pyspark.sql.functions import  explode

# set files location ADLS
files_location = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/api=state/"

df_state_json = spark.read.json(files_location)
# df_state_json.select('*', explode('*')).show()
df_state_json.printSchema()
display(df_state_json)

#### Method 2: Mount your storage account to your Databricks cluster

dbutils.fs.mount

In this section, you mount your Azure Data Lake Storage Gen2 cloud object storage to the Databricks File System (DBFS). You use the Azure AD service principle you created previously for authentication with the storage account. For more information, see Mounting cloud object storage on Azure Databricks.


In [0]:
configs = {f"fs.azure.account.key.{storage_account}.blob.core.windows.net": access_key}

dbutils.fs.mount(
source = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/api=state/",
mount_point = "/mnt/adls_test",
extra_configs = configs)

In [0]:
dbutils.fs.ls('/mnt/adls_test')

### refreshMounts command (dbutils.fs.refreshMounts)
Forces all machines in the cluster to refresh their mount cache, ensuring they receive the most recent information.

To display help for this command, run dbutils.fs.help("refreshMounts").



In [0]:
dbutils.fs.refreshMounts()

### unmount command (dbutils.fs.unmount)
Deletes a DBFS mount point.

 Warning

To avoid errors, never modify a mount point while other jobs are reading or writing to it. After modifying a mount, always run dbutils.fs.refreshMounts() on all other running clusters to propagate any mount updates. See refreshMounts command (dbutils.fs.refreshMounts).



In [0]:
dbutils.fs.unmount("/mnt/adls_test")